In [1]:
# import packages
import numpy as np
from numpy import random

import pandas as pd

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.linear_model import Lasso, LassoCV
from sklearn.preprocessing import PolynomialFeatures

import time

from sklearn.model_selection import KFold

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.saving import load_model

from abc import ABC, abstractmethod

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
# Why 0 GPUs

Num GPUs Available:  0


In [3]:
# set global seed
tf.keras.utils.set_random_seed(8953)

In [4]:
data = pd.read_csv('/Users/arberimbibaj/dataset_example_indicatorCATE.csv', header=None, index_col=[0])
data = data.to_numpy()

In [5]:
N = len(data)
d = len(data[0,:]) - 3

In [6]:
# train test split
random.shuffle(data)
training, test = data[:700,:], data[700:,:]

In [7]:
# slice dataset by treatment status
training_control = training[training[:,26]==0]
training_treatment = training[training[:,26]==1]

# slice test set by treatment status
test_control = test[test[:,26]==0]
test_treatment = test[test[:,26]==1]

# Y_train by treatment status
Y_train_control = training_control[:,0]
Y_train_treatment = training_treatment[:,0]

# Y_test by treatment status
Y_test_control = test_control[:,0]
Y_test_treatment = test_treatment[:,0]

# X_train by treatment status
X_train_control = training_control[:,1:26]
X_train_treatment = training_treatment[:,1:26]

# X_test by treatment status
X_test_control = test_control[:,1:26]
X_test_treatment = test_treatment[:,1:26]

# X and Y test
X_test = test[:,1:26]
Y_test = test[:,0]

# X_train and Y_train (no split by treatment status)
X_train = training[:,1:26]
Y_train = training[:,0]

# W_train and W_test
W_train = training[:,26]
W_test = test[:,26]

# tau_test
tau_test = test[:,27]
tau_test_control = test_control[:,27]
tau_test_treatment = test_treatment[:,27]

In [8]:
# set training and test features for the S-Learner (it views W as no different from other X's)
X_W_train = training[:,1:27]
X_W_test = test[:,1:27]
X_test_0 = np.concatenate((test[:,1:26],np.zeros((300,1))), axis=1)
X_test_1 = np.concatenate((test[:,1:26],np.ones((300,1))), axis=1)

In [ ]:
X_W_train

In [ ]:
X_train_control

In [ ]:
X_train_treatment

In [ ]:
Y_train_control

In [ ]:
Y_train_treatment

In [ ]:
X_train

In [ ]:
W_train

In [ ]:
Y_train

In [ ]:
X_test

In [ ]:
W_test

In [ ]:
Y_test

In [ ]:
tau_test

# T-Learner

In [ ]:
# T-Learner (example with Random Forest)

# mu_0
t_learner_mu0 = RandomForestRegressor(max_depth=100, random_state=0)
t_learner_mu0.fit(X_train_control,Y_train_control)
t_mu_0_hat = t_learner_mu0.predict(X_test)

# mu_1
t_learner_mu1 = RandomForestRegressor(max_depth=100, random_state=0)
t_learner_mu1.fit(X_train_treatment,Y_train_treatment)
t_mu_1_hat = t_learner_mu1.predict(X_test)
# Prediction = mu_1 - mu_0
t_tau_hat = t_mu_1_hat - t_mu_0_hat
t_tau_hat

In [ ]:
# mean squared error
((t_tau_hat - tau_test)**2).mean()

# S-Learner

In [ ]:
### S-Learner

In [ ]:
X_test_0

In [ ]:
X_test_1

In [ ]:
Y_train

In [ ]:
# S-learner (example with Random Forest)

# mu_x
s_learner = RandomForestRegressor(max_depth=100, random_state=0)
s_learner.fit(X_W_train,Y_train)

# mu_0_hat
s_mu_0_hat = s_learner.predict(X_test_0)

# mu_1_hat
s_mu_1_hat = s_learner.predict(X_test_1)

# tau_hat
s_tau_hat = s_mu_1_hat - s_mu_0_hat
s_tau_hat


In [ ]:
# mean squared error
((s_tau_hat - tau_test)**2).mean()

# X-Learner

In [ ]:
### X-Learner

# mu_0 (same procedure as for t-learner, maybe can speed up process)
x_learner_mu0 = RandomForestRegressor(max_depth=100, random_state=0)
x_learner_mu0.fit(X_train_control,Y_train_control)

# mu_1 (same procedure as for t-learner, maybe can speed up process)
x_learner_mu1 = RandomForestRegressor(max_depth=100, random_state=0)
x_learner_mu1.fit(X_train_treatment,Y_train_treatment)

# compute imputed treatment effect D_0 and D_1
# d_0
imputed_0 = x_learner_mu1.predict(X_train_control) - Y_train_control

# d_1
imputed_1 = Y_train_treatment - x_learner_mu0.predict(X_train_treatment)

# regress imputed on X
# tau_hat_0
x_tau_0_hat = RandomForestRegressor(max_depth=100, random_state=0)
x_tau_0_hat.fit(X_train_control ,imputed_0)

# tau_hat_1
x_tau_1_hat = RandomForestRegressor(max_depth=100, random_state=0)
x_tau_1_hat.fit(X_train_treatment ,imputed_1)

# estimate e_x to use as g_x
g_x_hat = RandomForestClassifier(max_depth=100, random_state=0)
g_x_hat.fit(X_train,W_train)
probabilities = g_x_hat.predict_proba(X_test)
probas_1 = probabilities[:,1]
probas_0 = probabilities[:,0]

# final estimator of tau
x_tau_hat = probas_1 * x_tau_0_hat.predict(X_test) + probas_0 * x_tau_1_hat.predict(X_test)
x_tau_hat


In [ ]:
# mean squared error (much lower here!)
((x_tau_hat - tau_test)**2).mean()

# R-Learner

In [ ]:
### R-Learner

# estimate e_x
r_learner_e_x = RandomForestClassifier(max_depth=100, random_state=0)
r_learner_e_x.fit(X_train,W_train)

# get e_x predictions
r_probas = r_learner_e_x.predict_proba(X_train)
r_probas_0 = r_probas[:,0] # probabilities of W=0
r_probas_1 = r_probas[:,1] # probabilities of W=1

# estimate mu_x
r_learner_mu_x = RandomForestRegressor(max_depth=100, random_state=0)
r_learner_mu_x.fit(X_train,Y_train)

# compute r-pseudo-outcome and weights
r_learner_pseudo_outcomes = (Y_train - r_learner_mu_x.predict(X_train)) / (W_train - r_probas_1)
r_learner_weights = (W_train - r_probas_1)**2

# estimate tau (regress pseudo-outcomes on X, weight by (W-e(x))^2)
r_learner_tau = RandomForestRegressor(max_depth=100, random_state=0)
r_learner_tau.fit(X_train,r_learner_pseudo_outcomes, sample_weight=r_learner_weights)

# predict tau
r_tau_hats = r_learner_tau.predict(X_test)
r_tau_hats

In [ ]:
((r_tau_hats - tau_test)**2).mean()

# DR-Learner

In [ ]:
### DR-Learner

# TODO: APPLY CROSS-FITTING?
# estimate e_x
dr_learner_e_x = RandomForestClassifier(n_estimators=100, max_depth=100, random_state=0)
dr_learner_e_x.fit(X_train, W_train)

dr_probas = dr_learner_e_x.predict_proba(X_train)
dr_probas_0 = dr_probas[:,0] # probabilities of W=0
dr_probas_1 = dr_probas[:,1] # probabilities of W=1

# estimate mu_0
dr_learner_mu_0 = RandomForestRegressor(n_estimators=100,max_depth=100, random_state=0)
dr_learner_mu_0.fit(X_train_control,Y_train_control)

# estimate mu_1
dr_learner_mu_1 = RandomForestRegressor(n_estimators=100,max_depth=100, random_state=0)
dr_learner_mu_1.fit(X_train_treatment,Y_train_treatment)

# DR-pseudo-outcomes
mu_w = W_train * dr_learner_mu_1.predict(X_train) + (1 - W_train) * dr_learner_mu_0.predict(X_train) # this is mu_w for each observation, i.e mu_1 for units in the treatment groups, and mu_0 for units in the control group
dr_pseudo_outcomes = (W_train - dr_probas_1) / (dr_probas_1 * dr_probas_0) * (Y_train - mu_w) + dr_learner_mu_1.predict(X_train) - dr_learner_mu_0.predict(X_train)

# estimate tau (regress pseudo-outcomes on X) # TODO: USE "Test Set" for this estimation
dr_learner_tau_hat = RandomForestRegressor(n_estimators=100,max_depth=100, random_state=0)
dr_learner_tau_hat.fit(X_train,dr_pseudo_outcomes)

# predict tau
dr_tau_hat = dr_learner_tau_hat.predict(X_test)
dr_tau_hat

In [ ]:
((dr_tau_hat - tau_test)**2).mean()

# RA-Learner

In [15]:
### RA-Learner

# mu_0 (same procedure as for t-learner, maybe can speed up process)
ra_learner_mu0 = RandomForestRegressor(max_depth=100, random_state=0)
ra_learner_mu0.fit(X_train_control,Y_train_control)

# mu_1 (same procedure as for t-learner, maybe can speed up process)
ra_learner_mu1 = RandomForestRegressor(max_depth=100, random_state=0)
ra_learner_mu1.fit(X_train_treatment,Y_train_treatment)

# e_x
ra_learner_e_x = RandomForestClassifier(max_depth=100, random_state=0)
ra_learner_e_x.fit(X_train,W_train)

# ra-pseudo-outcome
ra_pseudo_outcome = W_train*(Y_train - ra_learner_mu0.predict(X_train)) + (1 - W_train)*(ra_learner_mu1.predict(X_train) - Y_train)

# tau_hat
ra_tau_hat_learner = RandomForestRegressor(max_depth=100, random_state=0)
ra_tau_hat_learner.fit(X_train, ra_pseudo_outcome)
ra_tau_hat = ra_tau_hat_learner.predict(X_test)
ra_tau_hat

array([ 0.91490677,  8.26301955,  0.97106317,  3.5528225 ,  8.67564974,
       -4.68236172, 10.6688935 , -1.17413595, -2.63692004,  7.44641532,
        0.2918939 ,  6.43137586,  3.73575211,  2.59984884, -0.86203239,
        7.66755618,  2.51321221,  8.82989392, -4.41247407, -0.60076672,
        2.64336168,  1.99061936,  8.96845941,  7.7862815 ,  8.21040191,
        7.55267284, -1.14354599, -1.6653916 ,  1.25465375,  8.52964283,
       -0.46274891, -0.08576136,  0.63671972,  1.27520691, 11.51410645,
        2.00179904,  8.06064817,  0.79332432,  1.88817362,  0.45527881,
        1.06693595, -0.11435547,  2.77432839,  4.61230113,  8.27338701,
       10.53841517,  0.61909964, -4.58885619, -0.24858074,  1.83739071,
        7.30205278,  0.52086065,  9.27698339,  7.95976077,  7.9325154 ,
        8.00236788,  2.13039965, -2.13439482,  7.9438079 ,  1.6070221 ,
        7.90002368,  2.89252177, -2.14836991, -4.90215694,  8.36905704,
        7.70217215,  5.54715161, -1.83924182,  9.59718485,  6.96

In [16]:
# mean squared error
((ra_tau_hat - tau_test)**2).mean()

5.355494017645751

# PW-Learner

In [31]:
### PW-Learner
# mu_0 (same procedure as for t-learner, maybe can speed up process)
pw_learner_mu0 = RandomForestRegressor(max_depth=100, random_state=0)
pw_learner_mu0.fit(X_train_control,Y_train_control)

# mu_1 (same procedure as for t-learner, maybe can speed up process)
pw_learner_mu1 = RandomForestRegressor(max_depth=100, random_state=0)
pw_learner_mu1.fit(X_train_treatment,Y_train_treatment)

# e_x
pw_learner_e_x = RandomForestClassifier(max_depth=100, random_state=0)
pw_learner_e_x.fit(X_train,W_train)

# ra-pseudo-outcome
pw_pseudo_outcome = (W_train/pw_learner_e_x.predict_proba(X_train)[:,1] - (1 - W_train)/(pw_learner_e_x.predict_proba(X_train)[:,0]))*Y_train

# tau_hat
pw_tau_hat_learner = RandomForestRegressor(max_depth=100, random_state=0)
pw_tau_hat_learner.fit(X_train, pw_pseudo_outcome)
pw_tau_hat = pw_tau_hat_learner.predict(X_test)
pw_tau_hat

array([ 1.82258255e+00,  3.30253612e+00,  2.55699703e+00,  1.52583192e+00,
        3.05871242e+00, -7.93422335e+00,  1.66045421e+01,  6.15961297e-01,
       -4.57479980e+00,  2.83237001e+00, -1.72211845e+00,  2.29531910e+00,
        1.36240571e+00,  9.35168611e+00, -3.82059473e+00,  3.24921597e+00,
        4.72670705e+00,  2.61065262e+00,  4.19853116e+00, -1.09661101e+00,
        4.63329534e+00,  2.21814235e+00,  6.21359562e+00,  2.71285893e+00,
        7.53868582e+00,  1.72688984e+00,  4.69543777e-01, -6.14387931e-01,
       -5.41671550e+00, -1.00237007e+00, -2.05090451e+00,  4.79920008e+00,
        3.04292536e+00,  4.84487128e+00,  2.27057392e+00,  3.01363921e-01,
       -1.79640246e+00,  3.02498879e+00,  2.21134199e+00,  3.00075400e-01,
       -1.06077085e+01, -4.78677322e+00,  3.77265746e+00,  7.38518006e+00,
        3.09171163e+00,  8.09618033e+00, -1.83045696e+00, -7.97791039e+00,
        1.26115023e+00,  1.30512966e+00,  1.45183221e+00, -1.25497004e+01,
        2.64006377e+00, -

In [32]:
# mean squared error
((pw_tau_hat - tau_test)**2).mean()

30.529802728890644

# F-Learner: AKA THE SAME AS PW-LEARNER!!!

In [ ]:
### F-Learner
# mu_0 (same procedure as for t-learner, maybe can speed up process)
f_learner_mu0 = RandomForestRegressor(max_depth=100, random_state=0)
f_learner_mu0.fit(X_train_control,Y_train_control)

# mu_1 (same procedure as for t-learner, maybe can speed up process)
f_learner_mu1 = RandomForestRegressor(max_depth=100, random_state=0)
f_learner_mu1.fit(X_train_treatment,Y_train_treatment)

# e_x
f_learner_e_x = RandomForestClassifier(max_depth=100, random_state=0)
f_learner_e_x.fit(X_train,W_train)

# ra-pseudo-outcome
f_pseudo_outcome = (W_train/f_learner_e_x.predict_proba(X_train)[:,1] - (1 - W_train)/(f_learner_e_x.predict_proba(X_train)[:,0]))*Y_train

# tau_hat
f_tau_hat_learner = RandomForestRegressor(max_depth=100, random_state=0)
f_tau_hat_learner.fit(X_train, pw_pseudo_outcome)
f_tau_hat = f_tau_hat_learner.predict(X_test)
f_tau_hat

In [ ]:
# mean squared error
print(((f_tau_hat - tau_test)**2).mean())
print("Same as for PW-Learner")

# U-Learner

In [44]:
### U-Learner
# estimate e_x
u_learner_e_x = RandomForestClassifier(max_depth=100, random_state=0)
u_learner_e_x.fit(X_train,W_train)

# estimate mu_x
u_learner_mu_x = RandomForestRegressor(max_depth=100, random_state=0)
u_learner_mu_x.fit(X_train,Y_train)

# compute residuals
u_learner_residuals = (Y_train - u_learner_mu_x.predict(X_train))/(W_train - u_learner_e_x.predict_proba(X_train)[:,1])

# tau_hat - regress residuals on X
u_tau_hat_learner = RandomForestRegressor(max_depth=100, random_state=0)
u_tau_hat_learner.fit(X_train,u_learner_residuals)

u_tau_hats = u_tau_hat_learner.predict(X_test)
u_tau_hats


array([ 2.85250904e+00,  1.00547688e+01, -4.92574582e-01,  1.91452713e+00,
        7.92620400e+00, -1.16463139e+01,  1.31921917e+01, -2.11709957e+00,
       -7.16250459e+00,  6.00462123e+00,  2.72981362e+00,  7.66322441e+00,
        2.45976887e+00,  1.18104536e+00,  9.14107486e-01,  6.16511795e+00,
        1.08497420e+01,  1.01644602e+01, -2.16899051e+00, -1.81475606e+00,
        1.34111458e+00,  3.41936913e+00,  1.04168301e+01,  7.57884238e+00,
        9.50578740e+00,  2.09883177e+00,  1.69555973e+00,  2.79744550e+00,
        4.16811513e-03,  1.50075375e+01, -4.28915305e+00, -8.77925502e-01,
       -2.78016855e+00,  3.32467561e+00,  1.06499625e+01,  3.00485772e+00,
        8.31998895e+00,  4.89600153e+00,  6.81894459e-02,  2.19437557e-02,
       -1.49448971e+00, -1.36872242e+00,  4.83187150e+00,  9.35912173e+00,
        6.47748569e+00,  1.37123644e+01,  4.08149426e+00, -2.59808756e+00,
        3.33981472e+00,  1.33195860e+00,  7.00985818e+00,  2.88766820e+00,
        1.12951611e+01,  

In [45]:
# mean squared error
((u_tau_hats - tau_test)**2).mean()

30.921286420155806

# Just some lasso tests

In [ ]:
poly_train = PolynomialFeatures(degree=4, interaction_only=False, include_bias=False)
X_poly_train = poly_train.fit_transform(X_train)
poly_test = PolynomialFeatures(degree=4, interaction_only=False, include_bias=False)
X_poly_test = poly_test.fit_transform(X_test)
X_poly_train

In [ ]:
lasso_poly = LassoCV(cv=10, random_state=0, tol=1e-2)

In [ ]:
# degree 3: 10 seconds to fit (100% cpu)
# degree 4: 90 seconds to fit (100% cpu)

In [ ]:
y_predictions = lasso_poly.predict(X_poly_test)

In [ ]:
((y_predictions - Y_test)**2).mean()

# just some Neural Network test

In [ ]:
# make model
# 3 layers with 200 units (elu activation), 2 layers with 100 units (elu activations), 1 output layer (linear activation)
model = keras.Sequential([
    keras.Input(shape=(d,)),
    layers.Dense(units=200, activation="relu", name="layer1"),
    layers.Dense(units=200, activation="relu", name="layer2"),
    layers.Dense(units=200, activation="relu", name="layer3"),
    layers.Dense(units=100, activation="relu", name="layer4"),
    layers.Dense(units=100, activation="relu", name="layer5"),
    layers.Dense(units=1, activation="linear", name="layer6"),

], name="Dense_Neural_Network")
model.summary()

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(),  # Optimizer
    # Loss function to minimize
    loss=keras.losses.MeanSquaredError(),
    # List of metrics to monitor
    metrics=[keras.metrics.MeanSquaredError()],
)

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, start_from_epoch=0)

In [ ]:
print("Training Model")
training = model.fit(
    X_train,
    Y_train,
    batch_size=100,
    epochs=10,
    validation_data=(X_test, Y_test),
    callbacks=[callback] # include early stopping
)

In [ ]:
predictions

In [ ]:
print("Generate predictions for test samples")
predictions = np.reshape(model.predict(X_test),(300,))
print("predictions shape:", predictions.shape)
predictions

In [ ]:
results = model.evaluate(X_test, Y_test, batch_size=100)

In [ ]:
((predictions - Y_test)**2).mean()

# TRY SAME WITH LASSO AND NEURAL NETWORK!

## With Lasso (or L1-loss for logistic regression)

In [17]:
poly_train = PolynomialFeatures(degree=3, interaction_only=False, include_bias=False)
X_poly_train = poly_train.fit_transform(X_train)
poly_test = PolynomialFeatures(degree=3, interaction_only=False, include_bias=False)
X_poly_test = poly_test.fit_transform(X_test)
X_poly_train

array([[-4.17996000e-01, -2.58928000e-01,  4.39047000e-01, ...,
         8.77308859e-02, -5.72355474e-02,  3.73404172e-02],
       [-7.55952000e-01, -2.64981000e-01,  4.66614000e-01, ...,
        -8.53171876e-02, -4.85438862e-02, -2.76205645e-02],
       [-1.69104000e-01,  5.08556000e-01, -6.89860000e-01, ...,
        -4.17789894e-03, -3.62827576e-04, -3.15095821e-05],
       ...,
       [ 7.11108000e-01,  2.16507000e-01, -6.45339000e-01, ...,
         2.42391301e-01,  3.27017404e-01,  4.41189028e-01],
       [-1.60640000e-01,  6.23831000e-01, -3.81564000e-01, ...,
         3.10338596e-04, -1.69771130e-04,  9.28735159e-05],
       [ 4.43016000e-01, -6.36760000e-01,  5.13939000e-01, ...,
        -1.35528909e+00,  1.44432897e+00, -1.53921860e+00]])

In [18]:
# compute polynomial features for treatment and control groups in training set
poly_train_treatment = PolynomialFeatures(degree=3, interaction_only=False, include_bias=False)
X_poly_train_treatment = poly_train_treatment.fit_transform(X_train_treatment)
poly_train_control = PolynomialFeatures(degree=3, interaction_only=False, include_bias=False)
X_poly_train_control = poly_train_treatment.fit_transform(X_train_control)
poly_test = PolynomialFeatures(degree=3, interaction_only=False, include_bias=False)
X_poly_test = poly_test.fit_transform(X_test)

In [ ]:
X_poly_train_control

In [ ]:
X_poly_train_treatment


## T-learner Lasso

In [ ]:

# T-Learner (example with Lasso)
tic = time.perf_counter()
# mu_0
t_learner_mu0 = LassoCV(cv=10, tol=1e-2, random_state=0)
t_learner_mu0.fit(X_poly_train_control,Y_train_control)
t_mu_0_hat = t_learner_mu0.predict(X_poly_test)

# mu_1
t_learner_mu1 = LassoCV(cv=10, tol=1e-2, random_state=0)
t_learner_mu1.fit(X_poly_train_treatment,Y_train_treatment)
t_mu_1_hat = t_learner_mu1.predict(X_poly_test)

# Prediction = mu_1 - mu_0
t_tau_hat = t_mu_1_hat - t_mu_0_hat
toc = time.perf_counter()
t_tau_hat

In [ ]:
print(f'Time for calculations: {toc-tic}') # took 69 seconds

In [ ]:
((t_tau_hat - tau_test)**2).mean()

## S-learner Lasso

In [ ]:
# compute polynomial features for treatment and control groups in training set
xw_poly_train = PolynomialFeatures(degree=4, interaction_only=False, include_bias=False)
X_W_poly_train = poly_train_treatment.fit_transform(X_W_train)

xw_poly_test_0 = PolynomialFeatures(degree=4, interaction_only=False, include_bias=False)
X_poly_test_0 = xw_poly_test_0.fit_transform(X_test_0)

xw_poly_test_1 = PolynomialFeatures(degree=4, interaction_only=False, include_bias=False)
X_poly_test_1 = xw_poly_test_0.fit_transform(X_test_1)

In [ ]:
# S-learner (example with Random Forest)
tic = time.perf_counter()
# mu_x
s_learner = LassoCV(cv=10, tol=1e-2, random_state=0)
s_learner.fit(X_W_poly_train,Y_train)

# mu_0_hat
s_mu_0_hat = s_learner.predict(X_poly_test_0)

# mu_1_hat
s_mu_1_hat = s_learner.predict(X_poly_test_1)

# tau_hat
s_tau_hat = s_mu_1_hat - s_mu_0_hat
toc = time.perf_counter()
print(f'Time for computation: {toc-tic}')

In [ ]:
((s_tau_hat - tau_test)**2).mean()

## X-learner with lasso (or l1-penalty)
### TAKES A LOT OF TIME!

In [ ]:
### X-Learner

tic = time.perf_counter()

# mu_0 (same procedure as for t-learner, maybe can speed up process)
x_learner_mu0 = LassoCV(cv=10, tol=1, random_state=0)
x_learner_mu0.fit(X_poly_train_control,Y_train_control)

# mu_1 (same procedure as for t-learner, maybe can speed up process)
x_learner_mu1 = LassoCV(cv=10, tol=1, random_state=0)
x_learner_mu1.fit(X_poly_train_treatment,Y_train_treatment)

# compute imputed treatment effect D_0 and D_1
# d_0
imputed_0 = x_learner_mu1.predict(X_poly_train_control) - Y_train_control

# d_1
imputed_1 = Y_train_treatment - x_learner_mu0.predict(X_poly_train_treatment)

# regress imputed on X
# tau_hat_0
x_tau_0_hat = LassoCV(cv=10, tol=1, random_state=0)
x_tau_0_hat.fit(X_poly_train_control ,imputed_0)

# tau_hat_1
x_tau_1_hat = LassoCV(cv=10, tol=1, random_state=0)
x_tau_1_hat.fit(X_poly_train_treatment ,imputed_1)

# estimate e_x to use as g_x
g_x_hat = LogisticRegressionCV(cv=KFold(10), penalty='l1', solver='saga', tol=1, random_state=0)
g_x_hat.fit(X_poly_train,W_train)
probabilities = g_x_hat.predict_proba(X_poly_test)
probas_1 = probabilities[:,1]
probas_0 = probabilities[:,0]

# final estimator of tau
x_tau_hat = probas_1 * x_tau_0_hat.predict(X_poly_test) + probas_0 * x_tau_1_hat.predict(X_poly_test)

toc = time.perf_counter()

print(f'Time for computation: {toc-tic}') # 127 seconds

In [ ]:
((x_tau_hat - tau_test)**2).mean()

## R-learner with lasso (or l1-penalty)

In [ ]:
### R-Learner

tic = time.perf_counter()

# estimate e_x
r_learner_e_x = LogisticRegressionCV(cv=KFold(10), penalty='l1', solver='saga', tol=1, random_state=0)
r_learner_e_x.fit(X_poly_train,W_train)

# get e_x predictions
r_probas = r_learner_e_x.predict_proba(X_poly_train)
r_probas_0 = r_probas[:,0] # probabilities of W=0
r_probas_1 = r_probas[:,1] # probabilities of W=1

# estimate mu_x
r_learner_mu_x = LassoCV(cv=10, tol=1, random_state=0)
r_learner_mu_x.fit(X_poly_train,Y_train)

# compute r-pseudo-outcome and weights
r_learner_pseudo_outcomes = (Y_train - r_learner_mu_x.predict(X_poly_train)) / (W_train - r_probas_1)
r_learner_weights = (W_train - r_probas_1)**2

# estimate tau (regress pseudo-outcomes on X, weight by (W-e(x))^2)
r_learner_tau = LassoCV(cv=10, tol=1, random_state=0)
r_learner_tau.fit(X_poly_train,r_learner_pseudo_outcomes, sample_weight=r_learner_weights)

# predict tau
r_tau_hats = r_learner_tau.predict(X_poly_test)
r_tau_hats

toc = time.perf_counter()

print(f'Time for computation: {toc-tic} seconds') # 98 seconds

In [ ]:
((r_tau_hats - tau_test)**2).mean()

## Dr-learner with lasso (l1-penalty)

In [ ]:
### DR-Learner

tic = time.perf_counter()

# TODO: APPLY CROSS-FITTING?
# estimate e_x
dr_learner_e_x = LogisticRegressionCV(cv=KFold(10), penalty='l1', solver='saga', tol=1, random_state=0)
dr_learner_e_x.fit(X_poly_train, W_train)

dr_probas = dr_learner_e_x.predict_proba(X_poly_train)
dr_probas_0 = dr_probas[:,0] # probabilities of W=0
dr_probas_1 = dr_probas[:,1] # probabilities of W=1

# estimate mu_0
dr_learner_mu_0 = LassoCV(cv=10, tol=1, random_state=0)
dr_learner_mu_0.fit(X_poly_train_control,Y_train_control)

# estimate mu_1
dr_learner_mu_1 = LassoCV(cv=10, tol=1, random_state=0)
dr_learner_mu_1.fit(X_poly_train_treatment,Y_train_treatment)

# DR-pseudo-outcomes
mu_w = W_train * dr_learner_mu_1.predict(X_poly_train) + (1 - W_train) * dr_learner_mu_0.predict(X_poly_train) # this is mu_w for each observation, i.e mu_1 for units in the treatment groups, and mu_0 for units in the control group
dr_pseudo_outcomes = (W_train - dr_probas_1) / (dr_probas_1 * dr_probas_0) * (Y_train - mu_w) + dr_learner_mu_1.predict(X_poly_train) - dr_learner_mu_0.predict(X_poly_train)

# estimate tau (regress pseudo-outcomes on X) # TODO: USE "Test Set" for this estimation
dr_learner_tau_hat = LassoCV(cv=10, tol=1, random_state=0)
dr_learner_tau_hat.fit(X_poly_train,dr_pseudo_outcomes)

# predict tau
dr_tau_hat = dr_learner_tau_hat.predict(X_poly_test)

toc = time.perf_counter()

print(f'Time needed for computation: {toc-tic} seconds') # 104 seconds

In [ ]:
((dr_tau_hat - tau_test)**2).mean()

## Ra-learner with lasso

In [19]:
### RA-Learner

tic = time.perf_counter()

# mu_0 (same procedure as for t-learner, maybe can speed up process)
ra_learner_mu0 = LassoCV(cv=10, tol=1, random_state=0)
ra_learner_mu0.fit(X_poly_train_control,Y_train_control)

# mu_1 (same procedure as for t-learner, maybe can speed up process)
ra_learner_mu1 = LassoCV(cv=10, tol=1, random_state=0)
ra_learner_mu1.fit(X_poly_train_treatment,Y_train_treatment)

# e_x
ra_learner_e_x = LogisticRegressionCV(cv=KFold(10), penalty='l1', solver='saga', tol=1, random_state=0)
ra_learner_e_x.fit(X_poly_train,W_train)

# ra-pseudo-outcome
ra_pseudo_outcome = W_train*(Y_train - ra_learner_mu0.predict(X_poly_train)) + (1 - W_train)*(ra_learner_mu1.predict(X_poly_train) - Y_train)

# tau_hat
ra_tau_hat_learner = LassoCV(cv=10, tol=1, random_state=0)
ra_tau_hat_learner.fit(X_poly_train, ra_pseudo_outcome)
ra_tau_hat = ra_tau_hat_learner.predict(X_poly_test)

toc = time.perf_counter()

print(f'Time for computation: {toc-tic} seconds.') # 121 seconds

Time for computation: 16.094570000001113 seconds.


In [20]:
((ra_tau_hat - tau_test)**2).mean()

8.283890654355236

## PW-learner with lasso

In [33]:
### PW-Learner

tic = time.perf_counter()

# mu_0 (same procedure as for t-learner, maybe can speed up process)
pw_learner_mu0 = LassoCV(cv=10, tol=1, random_state=0)
pw_learner_mu0.fit(X_poly_train_control,Y_train_control)

# mu_1 (same procedure as for t-learner, maybe can speed up process)
pw_learner_mu1 = LassoCV(cv=10, tol=1, random_state=0)
pw_learner_mu1.fit(X_poly_train_treatment,Y_train_treatment)

# e_x
pw_learner_e_x = LogisticRegressionCV(cv=KFold(10), penalty='l1', solver='saga', tol=1, random_state=0)
pw_learner_e_x.fit(X_poly_train,W_train)

# ra-pseudo-outcome
pw_pseudo_outcome = (W_train/pw_learner_e_x.predict_proba(X_poly_train)[:,1] - (1 - W_train)/(pw_learner_e_x.predict_proba(X_poly_train)[:,0]))*Y_train

# tau_hat
pw_tau_hat_learner = LassoCV(cv=10, tol=1, random_state=0)
pw_tau_hat_learner.fit(X_poly_train, pw_pseudo_outcome)
pw_tau_hat = pw_tau_hat_learner.predict(X_poly_test)

toc = time.perf_counter()

print(f'Time for computation: {toc-tic} seconds.') # 117 seconds

Time for computation: 15.782583167005214 seconds.


In [34]:
((pw_tau_hat - tau_test)**2).mean()

16.03059004204301

## U-learner with lasso

In [46]:
### U-Learner

tic = time.perf_counter()

# estimate e_x
u_learner_e_x = LogisticRegressionCV(cv=KFold(10), penalty='l1', solver='saga', tol=1, random_state=0)
u_learner_e_x.fit(X_poly_train,W_train)

# estimate mu_x
u_learner_mu_x = LassoCV(cv=10, tol=1, random_state=0)
u_learner_mu_x.fit(X_poly_train,Y_train)

# compute residuals
u_learner_residuals = (Y_train - u_learner_mu_x.predict(X_poly_train))/(W_train - u_learner_e_x.predict_proba(X_poly_train)[:,1])

# tau_hat - regress residuals on X
u_tau_hat_learner = LassoCV(cv=10, tol=1, random_state=0)
u_tau_hat_learner.fit(X_poly_train,u_learner_residuals)

u_tau_hats = u_tau_hat_learner.predict(X_poly_test)

toc = time.perf_counter()

print(f'Time for computation: {toc-tic} seconds.') # 98 seconds


Time for computation: 14.17206100000476 seconds.


In [47]:
((u_tau_hats - tau_test)**2).mean()

7.6762472449663495

# now do it with neural networks


In [ ]:
# make model
# 3 layers with 200 units (elu activation), 2 layers with 100 units (elu activations), 1 output layer (linear activation)
model_25 = keras.Sequential([
    keras.Input(shape=(25,)),
    layers.Dense(units=200, activation="relu", name="layer1"),
    layers.Dense(units=200, activation="relu", name="layer2"),
    layers.Dense(units=200, activation="relu", name="layer3"),
    layers.Dense(units=100, activation="relu", name="layer4"),
    layers.Dense(units=100, activation="relu", name="layer5"),
    layers.Dense(units=1, activation="linear", name="layer6"),

], name="Dense_Neural_Network")

In [ ]:
# compile the model
model_25.compile(
    optimizer=keras.optimizers.Adam(),  # Optimizer
    # Loss function to minimize
    loss=keras.losses.MeanSquaredError(),
    # List of metrics to monitor
    metrics=[keras.metrics.MeanSquaredError()],
)

In [ ]:
# save the model
model_25.save('model_25')

In [ ]:
# same model, but input shape=26, for t-learner only
# 3 layers with 200 units (elu activation), 2 layers with 100 units (elu activations), 1 output layer (linear activation)
model_26 = keras.Sequential([
    keras.Input(shape=(26,)),
    layers.Dense(units=200, activation="relu", name="layer1"),
    layers.Dense(units=200, activation="relu", name="layer2"),
    layers.Dense(units=200, activation="relu", name="layer3"),
    layers.Dense(units=100, activation="relu", name="layer4"),
    layers.Dense(units=100, activation="relu", name="layer5"),
    layers.Dense(units=1, activation="linear", name="layer6"),

], name="Dense_Neural_Network")

In [ ]:
# compile the model
model_26.compile(
    optimizer=keras.optimizers.Adam(),  # Optimizer
    # Loss function to minimize
    loss=keras.losses.MeanSquaredError(),
    # List of metrics to monitor
    metrics=[keras.metrics.MeanSquaredError()],
)

In [ ]:
# save the model
model_26.save('model_26')

In [ ]:
model_ex = keras.Sequential([
    keras.Input(shape=(25,)),
    layers.Dense(units=200, activation="relu", name="layer1"),
    layers.Dense(units=200, activation="relu", name="layer2"),
    layers.Dense(units=200, activation="relu", name="layer3"),
    layers.Dense(units=100, activation="relu", name="layer4"),
    layers.Dense(units=100, activation="relu", name="layer5"),
    layers.Dense(units=1, activation="linear", name="layer6"),

], name="Dense_Neural_Network_Classification")

In [ ]:
# compile the model
model_ex.compile(
    optimizer=keras.optimizers.Adam(),  # Optimizer
    # Loss function to minimize
    loss=keras.losses.BinaryCrossentropy(from_logits=True, label_smoothing=0.5),
    # List of metrics to monitor
    metrics=keras.metrics.BinaryAccuracy(),
)

In [ ]:
# save the model
model_ex.save('model_ex')

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=0, start_from_epoch=30)

# Break

In [ ]:
model_ex_sigmoid = keras.Sequential([
    keras.Input(shape=(25,)),
    layers.Dense(units=200, activation="relu", name="layer1"),
    layers.Dense(units=200, activation="relu", name="layer2"),
    layers.Dense(units=200, activation="relu", name="layer3"),
    layers.Dense(units=100, activation="relu", name="layer4"),
    layers.Dense(units=100, activation="relu", name="layer5"),
    layers.Dense(units=1, activation="sigmoid", name="layer6"),

], name="Dense_Neural_Network_Classification")

In [ ]:
# compile the model
model_ex_sigmoid.compile(
    optimizer=keras.optimizers.Adam(),  # Optimizer
    # Loss function to minimize
    loss=keras.losses.BinaryCrossentropy(from_logits=False, label_smoothing=0.5),
    # List of metrics to monitor
    metrics=keras.metrics.BinaryAccuracy(),
)

In [ ]:
# save the model
model_ex_sigmoid.save('model_ex_sigmoid')

In [ ]:
# T-Learner (example with Random Forest)

tic = time.perf_counter()

# mu_0
t_learner_mu0 = load_model('model_25')
print('Training mu0')
t_learner_mu0.fit(X_train_control,Y_train_control,
    batch_size=100,
    epochs=100,
    validation_data=(X_test, Y_test),
    callbacks=None # include early stopping
)
t_mu_0_hat = t_learner_mu0.predict(X_test)

# mu_1
t_learner_mu1 = load_model('model_25')
print('Training mu1')
t_learner_mu1.fit(X_train_treatment,Y_train_treatment,
    batch_size=100,
    epochs=100,
    validation_data=(X_test, Y_test),
    callbacks=None # include early stopping
)
t_mu_1_hat = t_learner_mu1.predict(X_test)

# Prediction = mu_1 - mu_0
t_tau_hat = t_mu_1_hat - t_mu_0_hat

toc = time.perf_counter()

print(f'Time for computation: {toc-tic} seconds.') # 3 seconds


In [ ]:
((np.reshape(t_tau_hat,(300,)) - tau_test)**2).mean() # 3.18

## S-learner with NN

In [ ]:
# S-learner (example with Random Forest)

# mu_x
s_learner = load_model('model_26')
s_learner.fit(X_W_train,Y_train,
    batch_size=100,
    epochs=100,
    validation_data=(X_W_test, Y_test),
    callbacks=None # include early stopping
)

# mu_0_hat
s_mu_0_hat = s_learner.predict(X_test_0)

# mu_1_hat
s_mu_1_hat = s_learner.predict(X_test_1)

# tau_hat
s_tau_hat = s_mu_1_hat - s_mu_0_hat


In [ ]:
((np.reshape(s_tau_hat,(300,)) - tau_test)**2).mean() # 1.98

# X-learner with Neural Network

In [ ]:
### X-Learner

# mu_0 (same procedure as for t-learner, maybe can speed up process)
x_learner_mu0 = load_model('model_25')
x_learner_mu0.fit(X_train_control,Y_train_control,
    batch_size=100,
    epochs=100,
    validation_data=(X_test_control, Y_test_control),
    callbacks=None # include early stopping
)

# d_1
imputed_1 = Y_train_treatment - np.reshape(x_learner_mu0.predict(X_train_treatment),(len(Y_train_treatment),))

# mu_1 (same procedure as for t-learner, maybe can speed up process)
x_learner_mu1 = load_model('model_25')
x_learner_mu1.fit(X_train_treatment,Y_train_treatment,
    batch_size=100,
    epochs=100,
    validation_data=(X_test_treatment, Y_test_treatment),
    callbacks=None # include early stopping
)

# d_0
imputed_0 = np.reshape(x_learner_mu1.predict(X_train_control),(len(Y_train_control),)) - Y_train_control


# regress imputed on X

# tau_hat_1
x_tau_1_hat = load_model('model_25')
x_tau_1_hat.fit(X_train_treatment,imputed_1,
    batch_size=100,
    epochs=100,
    validation_data=(X_test_treatment, tau_test_treatment),
    callbacks=None # include early stopping
)

x_tau_1_hat_predicts = np.reshape(x_tau_1_hat.predict(X_test),(len(X_test),))

# tau_hat_0
x_tau_0_hat = load_model('model_25')
x_tau_0_hat.fit(X_train_control,imputed_0,
    batch_size=100,
    epochs=100,
    validation_data=(X_test_control, tau_test_control),
    callbacks=None # include early stopping
)

x_tau_0_hat_predicts = np.reshape(x_tau_0_hat.predict(X_test),(len(X_test),))

# estimate e_x to use as g_x
g_x_hat = load_model('model_ex')
g_x_hat.fit(X_train,W_train,
    batch_size=100,
    epochs=100,
    validation_data=(X_test, W_test),
    callbacks=None # include early stopping
)
x_probabilities = g_x_hat.predict(X_test)
x_probs_1 = np.reshape(keras.activations.sigmoid(x_probabilities),(len(x_probabilities,)))
x_probs_0 = 1 - x_probs_1

# final estimator of tau
x_tau_hat = x_probs_1 * x_tau_0_hat_predicts + x_probs_0 * x_tau_1_hat_predicts

In [ ]:
((np.reshape(x_tau_hat,(300,)) - tau_test)**2).mean() # 3.1614 with smoothing of 0.5

# R-learner with NN

In [ ]:
### R-Learner

# estimate e_x
r_learner_e_x = load_model('model_ex')
r_learner_e_x.fit(X_train,W_train,
    batch_size=100,
    epochs=100,
    validation_data=(X_test, W_test),
    callbacks=None # include early stopping
)

# get e_x predictions
r_probabilities = np.reshape(keras.activations.sigmoid(r_learner_e_x.predict(X_train)),len(X_train,))
r_probas_1 = r_probabilities# probabilities of W=1
r_probas_0 = 1 - r_probabilities # probabilities of W=0

# estimate mu_x
r_learner_mu_x = load_model('model_25')
r_learner_mu_x.fit(X_train,Y_train,
    batch_size=100,
    epochs=100,
    validation_data=(X_test, Y_test),
    callbacks=None # include early stopping
)

# compute r-pseudo-outcome and weights
r_learner_pseudo_outcomes = (Y_train - np.reshape(r_learner_mu_x.predict(X_train),(len(X_train),))) / (W_train - r_probas_1)
r_learner_weights = (W_train - r_probas_1)**2

# estimate tau (regress pseudo-outcomes on X, weight by (W-e(x))^2)
r_learner_tau = load_model('model_25')
r_learner_tau.fit(X_train,r_learner_pseudo_outcomes,
    sample_weight=r_learner_weights,
    batch_size=100,
    epochs=100,
    validation_data=None,
    callbacks=None # include early stopping
)

# predict tau
r_tau_hats = r_learner_tau.predict(X_test)

In [ ]:
((np.reshape(r_tau_hats,(len(X_test))) - tau_test)**2).mean() #47.81

It's reproducible now!

# DR-learner with Neural Network

In [ ]:
### DR-Learner

tic = time.perf_counter()

# TODO: APPLY CROSS-FITTING?
# estimate e_x
dr_learner_e_x = load_model('model_ex')
dr_learner_e_x.fit(X_train, W_train,
    batch_size=100,
    epochs=100,
    validation_data=(X_test, W_test),
    callbacks=None # include early stopping
)

dr_probabilities = np.reshape(keras.activations.sigmoid(dr_learner_e_x.predict(X_train)),len(X_train,))
dr_probas_0 = 1 - dr_probabilities # probabilities of W=0
dr_probas_1 = dr_probabilities # probabilities of W=1

# estimate mu_0
dr_learner_mu_0 = load_model('model_25')
dr_learner_mu_0.fit(X_train_control,Y_train_control,
    batch_size=100,
    epochs=100,
    validation_data=(X_test_control, Y_test_control),
    callbacks=None # include early stopping
)

dr_learner_mu_0_predictions = dr_learner_mu_0.predict(X_train)

# estimate mu_1
dr_learner_mu_1 = load_model('model_25')
dr_learner_mu_1.fit(X_train_treatment,Y_train_treatment,
    batch_size=100,
    epochs=100,
    validation_data=(X_test_treatment, Y_test_treatment),
    callbacks=None # include early stopping
)

dr_learner_mu_1_predictions = dr_learner_mu_1.predict(X_train)

# DR-pseudo-outcomes
mu_w = W_train * dr_learner_mu_1_predictions + (1 - W_train) * dr_learner_mu_0_predictions # this is mu_w for each observation, i.e mu_1 for units in the treatment groups, and mu_0 for units in the control group
dr_pseudo_outcomes = (W_train - dr_probas_1) / (dr_probas_1 * dr_probas_0) * (Y_train - mu_w) + dr_learner_mu_1_predictions - dr_learner_mu_0_predictions

# estimate tau (regress pseudo-outcomes on X) # TODO: USE "Test Set" for this estimation
dr_learner_tau_hat = load_model('model_25')
dr_learner_tau_hat.fit(X_train,dr_pseudo_outcomes,
    batch_size=100,
    epochs=100,
    validation_data=None,
    callbacks=None # include early stopping
)

# predict tau
dr_tau_hat = dr_learner_tau_hat.predict(X_test)

toc = time.perf_counter()

print(f'Time needed for computation: {toc-tic} seconds') # 104 seconds

In [ ]:
((np.reshape(dr_tau_hat,(len(tau_test),)) - tau_test)**2).mean() # 8.3514

# RA-learner with Neural Network

In [21]:
### RA-Learner

# mu_0 (same procedure as for t-learner, maybe can speed up process)
ra_learner_mu0 = load_model('model_25')
ra_learner_mu0.fit(X_train_control,Y_train_control,
    batch_size=100,
    epochs=100,
    validation_data=(X_test_control, Y_test_control),
    callbacks=None # include early stopping
)

# get hats
ra_learner_mu0_predictions = np.reshape(ra_learner_mu0.predict(X_train),(len(X_train),))

# mu_1 (same procedure as for t-learner, maybe can speed up process)
ra_learner_mu1 = load_model('model_25')
ra_learner_mu1.fit(X_train_treatment,Y_train_treatment,
    batch_size=100,
    epochs=100,
    validation_data=(X_test_treatment, Y_test_treatment),
    callbacks=None # include early stopping
)

# get hats
ra_learner_mu1_predictions = np.reshape(ra_learner_mu1.predict(X_train),(len(X_train),))


# e_x TODO: IS IT NEEDED?
"""ra_learner_e_x = load_model('model_ex')
ra_learner_e_x.fit(X_train,W_train,
    batch_size=100,
    epochs=100,
    validation_data=(X_test, W_test),
    callbacks=[callback] # include early stopping
)"""

# ra-pseudo-outcome
ra_pseudo_outcome = W_train*(Y_train - ra_learner_mu0_predictions) + (1 - W_train)*(ra_learner_mu1_predictions - Y_train)

# tau_hat
ra_tau_hat_learner = load_model('model_25')
ra_tau_hat_learner.fit(X_train, ra_pseudo_outcome,
    batch_size=100,
    epochs=100,
    validation_data=None,
    callbacks=None # include early stopping
)

ra_tau_hat = ra_tau_hat_learner.predict(X_test)
ra_tau_hat

Epoch 1/100
4/4 [==============================] - 0s 18ms/step - loss: 148.4175 - mean_squared_error: 148.4175 - val_loss: 144.6764 - val_mean_squared_error: 144.6764
Epoch 2/100
4/4 [==============================] - 0s 6ms/step - loss: 131.0420 - mean_squared_error: 131.0420 - val_loss: 113.7808 - val_mean_squared_error: 113.7808
Epoch 3/100
4/4 [==============================] - 0s 5ms/step - loss: 97.0849 - mean_squared_error: 97.0849 - val_loss: 61.1599 - val_mean_squared_error: 61.1599
Epoch 4/100
4/4 [==============================] - 0s 5ms/step - loss: 48.3961 - mean_squared_error: 48.3961 - val_loss: 45.0901 - val_mean_squared_error: 45.0901
Epoch 5/100
4/4 [==============================] - 0s 5ms/step - loss: 37.2469 - mean_squared_error: 37.2469 - val_loss: 32.6569 - val_mean_squared_error: 32.6569
Epoch 6/100
4/4 [==============================] - 0s 5ms/step - loss: 21.8929 - mean_squared_error: 21.8929 - val_loss: 26.7816 - val_mean_squared_error: 26.7816
Epoch 7/100
4

array([[ 2.1547678e+00],
       [ 7.0934086e+00],
       [-1.1933132e+00],
       [-1.7330331e+00],
       [ 4.1358261e+00],
       [ 3.0794033e-01],
       [ 5.6670022e+00],
       [-4.5303479e-01],
       [ 1.3761189e+00],
       [ 6.5797367e+00],
       [ 1.5958121e+00],
       [ 9.8061094e+00],
       [ 6.1213851e-01],
       [ 5.4904618e+00],
       [-5.1979393e-02],
       [ 8.1233473e+00],
       [ 4.4370487e-02],
       [ 8.1692820e+00],
       [ 1.0963433e+00],
       [-3.9844456e-01],
       [ 6.2407166e-01],
       [-2.8072139e-02],
       [ 9.1494427e+00],
       [ 7.8003716e+00],
       [ 9.3238668e+00],
       [ 5.3199363e+00],
       [ 2.6112435e+00],
       [ 2.7554927e+00],
       [ 8.6637962e-01],
       [ 8.1214581e+00],
       [ 7.6176071e-01],
       [-1.4713858e+00],
       [ 1.2543915e-01],
       [ 1.8318594e+00],
       [ 8.0083828e+00],
       [ 2.1588488e+00],
       [ 8.8536005e+00],
       [ 4.1732451e-01],
       [ 1.7001538e+00],
       [ 1.5187305e+00],


In [22]:
((np.reshape(ra_tau_hat,(len(tau_test),)) - tau_test)**2).mean() # 3.397

3.3973654461530867

# PW-learner with Neural Network

In [35]:
### PW-Learner
# mu_0 TODO: really needed
# pw_learner_mu0 = load_model('model_25')
"""pw_learner_mu0.fit(X_train_control,Y_train_control,
    batch_size=100,
    epochs=100,
    validation_data=(X_test_control, Y_test_control),
    callbacks=None # include early stopping
)"""

# mu_1 TODO: really needed?
#pw_learner_mu1 = load_model('model_25')
"""pw_learner_mu1.fit(X_train_treatment,Y_train_treatment,
    batch_size=100,
    epochs=100,
    validation_data=(X_test_treatment, Y_test_treatment),
    callbacks=None # include early stopping
)"""

# e_x
pw_learner_e_x = load_model('model_ex')
pw_learner_e_x.fit(X_train,W_train,
    batch_size=100,
    epochs=100,
    validation_data=(X_test, W_test),
    callbacks=None # include early stopping
)

pw_probabilities = np.reshape(keras.activations.sigmoid(pw_learner_e_x.predict(X_train)),len(X_train,))
pw_probs_1 = pw_probabilities
pw_probs_0 = 1 - pw_probabilities

# ra-pseudo-outcome
pw_pseudo_outcome = (W_train / pw_probs_1 - (1 - W_train) / pw_probs_0)*Y_train

# tau_hat
pw_tau_hat_learner = load_model('model_25')
pw_tau_hat_learner.fit(X_train, pw_pseudo_outcome,
    batch_size=100,
    epochs=100,
    validation_data=None,
    callbacks=None # include early stopping
)
pw_tau_hat = pw_tau_hat_learner.predict(X_test)
pw_tau_hat

Epoch 1/100
7/7 [==============================] - 0s 11ms/step - loss: 0.6941 - binary_accuracy: 0.4971 - val_loss: 0.6991 - val_binary_accuracy: 0.4833
Epoch 2/100
7/7 [==============================] - 0s 3ms/step - loss: 0.6898 - binary_accuracy: 0.5000 - val_loss: 0.7003 - val_binary_accuracy: 0.4833
Epoch 3/100
7/7 [==============================] - 0s 3ms/step - loss: 0.6887 - binary_accuracy: 0.4986 - val_loss: 0.7013 - val_binary_accuracy: 0.4833
Epoch 4/100
7/7 [==============================] - 0s 3ms/step - loss: 0.6875 - binary_accuracy: 0.5071 - val_loss: 0.7057 - val_binary_accuracy: 0.4767
Epoch 5/100
7/7 [==============================] - 0s 3ms/step - loss: 0.6842 - binary_accuracy: 0.5029 - val_loss: 0.7039 - val_binary_accuracy: 0.4800
Epoch 6/100
7/7 [==============================] - 0s 3ms/step - loss: 0.6806 - binary_accuracy: 0.5143 - val_loss: 0.7067 - val_binary_accuracy: 0.4800
Epoch 7/100
7/7 [==============================] - 0s 3ms/step - loss: 0.6782 - b

array([[ 3.64949203e+00],
       [ 2.68186867e-01],
       [ 5.46157551e+00],
       [ 1.00083704e+01],
       [ 8.89912128e+00],
       [ 1.21294508e+01],
       [-2.65738220e+01],
       [-1.24039388e+00],
       [-1.44171696e+01],
       [-1.27449636e+01],
       [ 8.80474949e+00],
       [ 5.65772200e+00],
       [ 1.80343723e+00],
       [ 4.48548927e+01],
       [-1.65572357e+01],
       [ 2.13477573e+01],
       [ 8.48676586e+00],
       [ 2.41762218e+01],
       [ 1.37603703e+01],
       [-1.65333402e+00],
       [ 3.22607231e+01],
       [ 1.16547155e+01],
       [ 4.15380669e+00],
       [ 7.91893291e+00],
       [-6.72032022e+00],
       [-2.16316280e+01],
       [-1.15251217e+01],
       [ 1.70729294e+01],
       [-1.37344446e+01],
       [ 8.47141361e+00],
       [-3.37798524e+00],
       [ 5.92239666e+00],
       [ 1.19024897e+01],
       [-3.11693883e+00],
       [ 8.01198006e+00],
       [ 3.57970452e+00],
       [-4.39426756e+00],
       [ 5.65015221e+00],
       [ 1.9

In [36]:
((np.reshape(pw_tau_hat,(len(tau_test),)) - tau_test)**2).mean() # 271.842 TODO: CHECK IF IT REALLY IS CORRECT

271.8425349295992

# U-learner with Neural Network

In [63]:
### U-Learner
# estimate e_x
u_learner_e_x = load_model('model_ex')
u_learner_e_x.fit(X_train,W_train,
    batch_size=100,
    epochs=100,
    validation_data=(X_test, W_test),
    callbacks=None # include early stopping
)

u_probs_1 = np.reshape(u_learner_e_x.predict(X_train),(len(X_train),))
u_probs_0 = 1 - u_probs_1

# estimate mu_x
u_learner_mu_x = load_model('model_25')
u_learner_mu_x.fit(X_train,Y_train,
    batch_size=100,
    epochs=100,
    validation_data=(X_test,Y_test),
    callbacks=None
)

u_learner_mu_x_predictions = np.reshape(u_learner_mu_x.predict(X_train),(len(X_train),))
# compute residuals
u_learner_residuals = (Y_train - u_learner_mu_x_predictions)/(W_train - u_probs_1)

# tau_hat - regress residuals on X
u_tau_hat_learner = load_model('model_25')
u_tau_hat_learner.fit(X_train,u_learner_residuals,
    batch_size=100,
    epochs=100,
    validation_data=None,
    callbacks=None
)

u_tau_hats = u_tau_hat_learner.predict(X_test)
u_tau_hats


Epoch 1/100
7/7 [==============================] - 0s 10ms/step - loss: 0.6941 - binary_accuracy: 0.4971 - val_loss: 0.6991 - val_binary_accuracy: 0.4833
Epoch 2/100
7/7 [==============================] - 0s 3ms/step - loss: 0.6898 - binary_accuracy: 0.5000 - val_loss: 0.7003 - val_binary_accuracy: 0.4833
Epoch 3/100
7/7 [==============================] - 0s 3ms/step - loss: 0.6887 - binary_accuracy: 0.4986 - val_loss: 0.7013 - val_binary_accuracy: 0.4833
Epoch 4/100
7/7 [==============================] - 0s 3ms/step - loss: 0.6875 - binary_accuracy: 0.5071 - val_loss: 0.7057 - val_binary_accuracy: 0.4767
Epoch 5/100
7/7 [==============================] - 0s 3ms/step - loss: 0.6842 - binary_accuracy: 0.5029 - val_loss: 0.7039 - val_binary_accuracy: 0.4800
Epoch 6/100
7/7 [==============================] - 0s 3ms/step - loss: 0.6806 - binary_accuracy: 0.5143 - val_loss: 0.7067 - val_binary_accuracy: 0.4800
Epoch 7/100
7/7 [==============================] - 0s 3ms/step - loss: 0.6782 - b

array([[ 4.08133793e+00],
       [-9.51504529e-01],
       [-1.79076290e+00],
       [-7.63607311e+00],
       [-3.14084358e+01],
       [-2.24210358e+00],
       [-1.06579027e+01],
       [-1.28826427e+00],
       [-3.51480579e+00],
       [ 2.12843063e+02],
       [ 9.24310744e-01],
       [-1.22622738e+01],
       [-1.18085003e+00],
       [-3.41351986e+00],
       [-2.83067417e+00],
       [-5.83536301e+01],
       [-2.83002377e+00],
       [ 6.36574173e+01],
       [-1.04902945e+01],
       [ 1.65513083e-01],
       [-3.58274841e+00],
       [-4.25617158e-01],
       [-9.52702141e+00],
       [-6.10580730e+00],
       [-4.23984945e-01],
       [-3.26232338e+01],
       [-9.23550701e+00],
       [ 6.32225096e-01],
       [ 6.16047442e-01],
       [-5.90938873e+01],
       [-1.61812449e+00],
       [-2.58650589e+00],
       [ 1.12865295e+01],
       [-9.62632847e+00],
       [ 2.36428680e+01],
       [-5.57943344e+00],
       [-1.28381548e+01],
       [-4.89553356e+00],
       [-3.1

In [57]:
((np.reshape(u_tau_hats,(len(tau_test),)) - tau_test)**2).mean() # 74893.80 TODO: CHECK IF IT REALLY IS CORRECT!!!

74893.80292670502

###### STABLE ESTIMATOR? SINCE EXTREME VALUES FOR ESTIMATED PROBABILITIES!

# Make Classes

In [ ]:
class Metalearner():
    pass

# Class T Learner

In [ ]:
class TLearner(): # TODO: comment what is what.
    def __init__(self, method): # TODO: or maybe not give base_learners but method, i.e. : 'lasso', 'rf' or 'nn'
        self.method = method

        if method == 'rf':
            self.mu0_model = RandomForestRegressor(n_estimators=1000, max_depth=100, random_state=0)
            self.mu1_model = RandomForestRegressor(n_estimators=1000, max_depth=100, random_state=0)
        elif method == 'lasso':
            self.mu0_model = LassoCV(cv=10, tol=1e-2, random_state=0)
            self.mu1_model = LassoCV(cv=10, tol=1e-2, random_state=0)
            self.poly = PolynomialFeatures(degree=3, interaction_only=False, include_bias=False)
        elif method == 'nn':
            self.mu0_model = load_model('model_25')
            self.mu1_model = load_model('model_25')
        else:
            raise NotImplementedError('Base learner method not specified')



    def fit(self,
            x, y, w):# TODO: training process
        if self.method == 'rf':
            # 1: train mu_0
            print("Fitting random forest for mu_0")
            self.mu0_model.fit(x[w==0], y[w==0])

            # 2: train mu_1
            print("Fitting random forest for mu_1")
            self.mu1_model.fit(x[w==1], y[w==1])

        elif self.method == 'lasso':
            # make polynomial features
            x_poly_train = self.poly.fit_transform(x)

            # 1: train mu_0
            print("Fitting lasso for mu_0")
            self.mu0_model.fit(x_poly_train[w==0], y[w==0])

            # 2: train mu_1
            print("Fitting lasso for mu_1")
            self.mu1_model.fit(x_poly_train[w==1], y[w==1])

        elif self.method == 'nn':
            # 1: train mu_0
            print("Training neural network for mu_0")
            self.mu0_model.fit(x[w==0], y[w==0],
                           batch_size=100,
                           epochs=100,
                           validation_data=(X_test_control, Y_test_control),
                           callbacks=None, # include early stopping
                           verbose=0
                               )

            # 2: train mu_1
            print("Training neural network for mu_1")
            self.mu1_model.fit(x[w==1], y[w==1],
                           batch_size=100,
                           epochs=100,
                           validation_data=(X_test_treatment, Y_test_treatment),
                           callbacks=None, # include early stopping
                           verbose=0
                               )

        else:
            raise NotImplementedError('Base learner method not specified in fit')



    def predict(self,
                x): # TODO:
        if self.method == 'rf':
            # 1: calculate hats of mu_1 & mu_0
            mu0_hats = self.mu0_model.predict(x)
            mu1_hats = self.mu1_model.predict(x)
            predictions = mu1_hats - mu0_hats

        elif self.method == 'lasso':
            # make polynomial features
            x_poly_test = self.poly.fit_transform(x)

            # 1: calculate hats of mu_1 & mu_0
            mu0_hats = self.mu0_model.predict(x_poly_test)
            mu1_hats = self.mu1_model.predict(x_poly_test)
            predictions = mu1_hats - mu0_hats

        elif self.method == 'nn':
            mu0_hats = self.mu0_model.predict(x, verbose=0)
            mu1_hats = self.mu1_model.predict(x, verbose=0)
            predictions = np.reshape(mu1_hats-mu0_hats,(len(x),))

        else:
            raise NotImplementedError('Base learner method not specified in predict')
        return predictions

In [ ]:
t_nn = TLearner(method='lasso')
t_nn.fit(X_train, Y_train, W_train)
predictions = t_nn.predict(X_test)
((predictions - tau_test)**2).mean()
# rf: 10.091322530886687
# lasso: 5.583461099392904
# nn: 3.1867804239471273

# Class S Learner

In [ ]:
class SLearner(): # TODO: comment what is what.
    def __init__(self, method): # TODO: or maybe not give base_learners but method, i.e. : 'lasso', 'rf' or 'nn'
        self.method = method

        if method == 'rf':
            self.mux_model = RandomForestRegressor(n_estimators=1000, max_depth=100, random_state=0)
        elif method == 'lasso':
            self.mux_model = LassoCV(cv=10, tol=1e-2, random_state=0)
            self.poly = PolynomialFeatures(degree=3, interaction_only=False, include_bias=False)
        elif method == 'nn':
            self.mux_model = load_model('model_26')
        else:
            raise NotImplementedError('Base learner method not specified')



    def fit(self,
            x, y, w): # TODO: training process
        x_w = np.concatenate((x,np.reshape(w,(len(w),1))), axis=1)

        if self.method == 'rf':
            # 1: train mu_x
            print("Fitting random forest for mu_x")
            self.mux_model.fit(x_w, y)

        elif self.method == 'lasso':
            # make polynomial features
            x_poly_train = self.poly.fit_transform(x_w)

            # 1: train mu_x
            print("Fitting lasso for mu_x")
            self.mux_model.fit(x_poly_train, y)


        elif self.method == 'nn':
            # 1: train mu_x
            print("Training neural network for mu_x")
            self.mux_model.fit(x_w, y,
                           batch_size=100,
                           epochs=100,
                           callbacks=None, # include early stopping
                           verbose=0
                               )

        else:
            raise NotImplementedError('Base learner method not specified in fit')



    def predict(self,
                x, w): # TODO:
        x_0 = np.concatenate((x,np.zeros((len(x),1))), axis=1)
        x_1 = np.concatenate((x,np.ones((len(x),1))), axis=1)

        if self.method == 'rf':
            # 1: calculate hats of mu_x with X and W=1 or W=0
            mu0_hats = self.mux_model.predict(x_0)
            mu1_hats = self.mux_model.predict(x_1)
            predictions = mu1_hats - mu0_hats

        elif self.method == 'lasso':
            # make polynomial features
            x_poly_0 = self.poly.fit_transform(x_0)
            x_poly_1 = self.poly.fit_transform(x_1)

            # 1: calculate hats of mu_x with X and W=1 or W=0
            mu0_hats = self.mux_model.predict(x_poly_0)
            mu1_hats = self.mux_model.predict(x_poly_1)
            predictions = mu1_hats - mu0_hats

        elif self.method == 'nn':
            # 1: calculate hats of mu_x with X and W=1 or W=0
            mu0_hats = self.mux_model.predict(x_0, verbose=0)
            mu1_hats = self.mux_model.predict(x_1, verbose=0)
            predictions = np.reshape(mu1_hats-mu0_hats,(len(x),))

        else:
            raise NotImplementedError('Base learner method not specified in predict')
        return predictions

In [ ]:
s_nn = SLearner('lasso')
s_nn.fit(X_train, Y_train, W_train)
predictions = s_nn.predict(X_test, W_test)
((predictions - tau_test)**2).mean()
# rf: 18.134009488483855
# lasso: 5.559126710289806
# nn: 1.987529792077956

# Class X Learner

In [ ]:
class XLearner(): # TODO: comment what is what.
    def __init__(self, method): # TODO: or maybe not give base_learners but method, i.e. : 'lasso', 'rf' or 'nn'
        self.method = method

        if method == 'rf':
            self.mu0_model = RandomForestRegressor(n_estimators=100, max_depth=100, random_state=0)
            self.mu1_model = RandomForestRegressor(n_estimators=100, max_depth=100, random_state=0)
            self.ex_model = RandomForestClassifier(max_depth=100, random_state=0)
            self.tau0_model = RandomForestRegressor(n_estimators=100, max_depth=100, random_state=0)
            self.tau1_model = RandomForestRegressor(n_estimators=100, max_depth=100, random_state=0)

        elif method == 'lasso':
            self.mu0_model = LassoCV(cv=10, tol=1, random_state=0)
            self.mu1_model = LassoCV(cv=10, tol=1, random_state=0)
            self.ex_model = LogisticRegressionCV(cv=KFold(10), penalty='l1', solver='saga', tol=1, random_state=0)
            self.tau0_model = LassoCV(cv=10, tol=1, random_state=0)
            self.tau1_model = LassoCV(cv=10, tol=1, random_state=0)
            self.poly = PolynomialFeatures(degree=3, interaction_only=False, include_bias=False)

        elif method == 'nn':
            self.mu0_model = load_model('model_25')
            self.mu1_model = load_model('model_25')
            self.ex_model = load_model('model_ex')
            self.tau0_model = load_model('model_25')
            self.tau1_model = load_model('model_25')
        else:
            raise NotImplementedError('Base learner method not specified')



    def fit(self,
            x, y, w):# TODO: training process
        if self.method == 'rf':
            # 1: train mu_0 and get imputed_1
            print("Fitting random forest for mu_0")
            self.mu0_model.fit(x[w==0], y[w==0])
            imputed_1 = y[w==1] - self.mu0_model.predict(x[w==1])

            # 2: train mu_1 and get imputed_0
            print("Fitting random forest for mu_1")
            self.mu1_model.fit(x[w==1], y[w==1])
            imputed_0 = self.mu1_model.predict(x[w==0]) - y[w==0]

            # 3: train tau_0
            print("Fitting random forest for tau_0")
            self.tau0_model.fit(x[w==0], imputed_0)

            # 4: train tau_1
            print("Fitting random forest for tau_1")
            self.tau1_model.fit(x[w==1], imputed_1)

            # 5: train e_x
            print("Fitting random forest for e_x")
            self.ex_model.fit(x,w)

        elif self.method == 'lasso':
            # make polynomial features
            x_poly_train = self.poly.fit_transform(x)

            # 1: train mu_0 and get imputed_1
            print("Fitting lasso for mu_0")
            self.mu0_model.fit(x_poly_train[w==0], y[w==0])
            imputed_1 = y[w==1] - self.mu0_model.predict(x_poly_train[w==1])


            # 2: train mu_1 and get imputed_0
            print("Fitting lasso for mu_1")
            self.mu1_model.fit(x_poly_train[w==1], y[w==1])
            imputed_0 = self.mu1_model.predict(x_poly_train[w==0]) - y[w==0]

            # 3: train tau_0
            print("Fitting random forest for tau_0")
            self.tau0_model.fit(x_poly_train[w==0], imputed_0)

            # 4: train tau_1
            print("Fitting random forest for tau_1")
            self.tau1_model.fit(x_poly_train[w==1], imputed_1)

            # 5: train e_x
            print("Fitting random forest for e_x")
            self.ex_model.fit(x_poly_train,w)

        elif self.method == 'nn':
            # 1: train mu_0
            print("Training neural network for mu_0")
            self.mu0_model.fit(x[w==0], y[w==0],
                               batch_size=100,
                               epochs=100,
                               callbacks=None, # include early stopping
                               verbose=0
                               )
            imputed_1 = y[w==1] - np.reshape(self.mu0_model.predict(x[w==1], verbose=0),(len(x[w==1]),))

            # 2: train mu_1
            print("Training neural network for mu_1")
            self.mu1_model.fit(x[w==1], y[w==1],
                               batch_size=100,
                               epochs=100,
                               callbacks=None, # include early stopping
                               verbose=0
                               )
            imputed_0 = np.reshape(self.mu1_model.predict(x[w==0], verbose=0),(len(x[w==0]),)) - y[w==0]

            # 3: train tau_0
            print("Fitting random forest for tau_0")
            self.tau0_model.fit(x[w==0], imputed_0,
                                batch_size=100,
                                epochs=100,
                                callbacks=None, # include early stopping
                                verbose=0
                                )

            # 4: train tau_1
            print("Fitting random forest for tau_1")
            self.tau1_model.fit(x[w==1], imputed_1,
                                batch_size=100,
                                epochs=100,
                                callbacks=None, # include early stopping
                                verbose=0
                                )

            # 5: train e_x
            print("Fitting random forest for e_x")
            self.ex_model.fit(x,w,
                              batch_size=100,
                              epochs=100,
                              callbacks=None, # include early stopping
                              verbose=0
                              )

        else:
            raise NotImplementedError('Base learner method not specified in fit')



    def predict(self,
                x): # TODO:
        if self.method == 'rf':
            # 1: calculate hats of tau_0 and tau_1
            tau_0_hats = self.tau0_model.predict(x)
            tau_1_hats = self.tau1_model.predict(x)
            # 2: probabilities
            probs = self.ex_model.predict_proba(x)[:,1]
            # 3: final predictions

        elif self.method == 'lasso':
            # make polynomial features
            x_poly_test = self.poly.fit_transform(x)

            # 1: calculate hats of tau_0 and tau_1
            tau_0_hats = self.tau0_model.predict(x_poly_test)
            tau_1_hats = self.tau1_model.predict(x_poly_test)
            probs = self.ex_model.predict_proba(x_poly_test)[:,1]

        elif self.method == 'nn':
            # 1: calculate hats of tau_0 and tau_1
            tau_0_hats = np.reshape(self.tau0_model.predict(x, verbose=0),(len(x),))
            tau_1_hats = np.reshape(self.tau1_model.predict(x, verbose=0),(len(x),))
            # 2: probabilities
            logit = self.ex_model.predict(x, verbose=0)
            probs = np.reshape(keras.activations.sigmoid(logit),(len(logit,)))
            # 3: final predictions

        else:
            raise NotImplementedError('Base learner method not specified in predict')

        predictions = probs * tau_0_hats + (1 - probs) * tau_1_hats
        return predictions

In [ ]:
x_rf = XLearner('nn')
x_rf.fit(X_train, Y_train, W_train)
predictions = x_rf.predict(X_test)
((predictions - tau_test)**2).mean()
# rf: # 3.1369636408859614 --> same (good)
# lasso: # nn: 7.667219448077926 --> same (good)
# nn: 3.161416602361538 --> same (good)

# R-Learner

In [ ]:
class RLearner():
    def __init__(self, method):
        self.method = method

        if method == 'rf':
            self.mux_model = RandomForestRegressor(n_estimators=100, max_depth=100, random_state=0)
            self.ex_model = RandomForestClassifier(max_depth=100, random_state=0)
            self.tau_model = RandomForestRegressor(n_estimators=100, max_depth=100, random_state=0)

        elif method == 'lasso':
            self.mux_model = LassoCV(cv=10, tol=1, random_state=0)
            self.ex_model = LogisticRegressionCV(cv=KFold(10), penalty='l1', solver='saga', tol=1, random_state=0)
            self.tau_model = LassoCV(cv=10, tol=1, random_state=0)
            self.poly = PolynomialFeatures(degree=3, interaction_only=False, include_bias=False)

        elif method == 'nn':
            self.mux_model = load_model('model_25')
            self.ex_model = load_model('model_ex')
            self.tau_model = load_model('model_25')

        else:
            raise NotImplementedError('Base learner method not specified or typo')

    def fit(self, x, y, w):

        if self.method == 'rf':
            # 1: fit mu_x
            print('Fitting random forest for mu_x')
            self.mux_model.fit(x,y)

            print('Fitting random forest for e_x')
            # 2: fit ex
            self.ex_model.fit(x,w)

            # 3: calculate pseudo_outcomes & weights
            probs = self.ex_model.predict_proba(x)[:,1]
            pseudo_outcomes = (y - self.mux_model.predict(x)) / (w - probs)
            weights = (w - probs)**2

            print('Fitting random forest for tau_x')
            # 4: fit tau
            self.tau_model.fit(x,pseudo_outcomes, sample_weight=weights)

        elif self.method == 'lasso':
            x_poly_train = self.poly.fit_transform(x)

            # 1: fit mu_x
            print('Fitting lasso for mu_x')
            self.mux_model.fit(x_poly_train,y)

            # 2: fit ex
            print('Fitting lasso for e_x')
            self.ex_model.fit(x_poly_train,w)

            # 3: calculate pseudo_outcomes & weights
            probs = self.ex_model.predict_proba(x_poly_train)[:,1]
            pseudo_outcomes = (y - self.mux_model.predict(x_poly_train)) / (w - probs)
            weights = (w - probs)**2

            # 4: fit tau
            print('Fitting lasso for tau_x')
            self.tau_model.fit(x_poly_train,pseudo_outcomes, sample_weight=weights)

        elif self.method == 'nn':

            # 1: fit mu_x
            print('Training NN for mu_x')
            self.mux_model.fit(x,y,
                               batch_size=100,
                               epochs=100,
                               callbacks=None,
                               verbose=0
                               )
            # 2: fit ex
            print('Training NN for e_x')
            self.ex_model.fit(x,w,
                              batch_size=100,
                              epochs=100,
                              callbacks=None,
                              verbose=0
                              )

            # 3: calculate pseudo_outcomes & weights
            probs = np.reshape(keras.activations.sigmoid(self.ex_model.predict(x, verbose=0)),len(x,))
            pseudo_outcomes = (y - np.reshape(self.mux_model.predict(x, verbose=0),(len(x),))) / (w - probs)
            weights = (w - probs)**2

            # 4: fit tau
            print('Training NN for tau_x')
            self.tau_model.fit(x,pseudo_outcomes,
                               sample_weight=weights,
                               batch_size=100,
                               epochs=100,
                               validation_data=None,
                               callbacks=None,
                               verbose=0
                               )

        else:
            raise NotImplementedError('Base learner method not specified in fit')

    def predict(self, x):

        if self.method == 'rf':
            predictions = self.tau_model.predict(x)

        elif self.method == 'lasso':
            x_poly_test = self.poly.fit_transform(x)
            predictions = self.tau_model.predict(x_poly_test)

        elif self.method == 'nn':
            predictions = np.reshape(self.tau_model.predict(x, verbose=0),(len(x),))

        else:
            raise NotImplementedError('Base learner method not specified in predict')

        return predictions

In [ ]:
r_rf = RLearner('nn')
r_rf.fit(X_train, Y_train, W_train)
predictions = r_rf.predict(X_test)
((predictions - tau_test)**2).mean()
# rf: 17.722925118749608
# lasso: 5.50038865455844
# nn: 47.81939839016621

# Class DR-Learner

In [ ]:
class DRLearner():
    def __init__(self, method):
        self.method = method
        if method == 'rf':
            self.mu0_model = RandomForestRegressor(n_estimators=100, max_depth=100, random_state=0)
            self.mu1_model = RandomForestRegressor(n_estimators=100, max_depth=100, random_state=0)
            self.ex_model = RandomForestClassifier(n_estimators=100, max_depth=100, random_state=0)
            self.tau_model = RandomForestRegressor(n_estimators=100, max_depth=100, random_state=0)

        elif method == 'lasso':
            self.mu0_model = LassoCV(cv=10, tol=1, random_state=0)
            self.mu1_model = LassoCV(cv=10, tol=1, random_state=0)
            self.ex_model = LogisticRegressionCV(cv=KFold(10), penalty='l1', solver='saga', tol=1, random_state=0)
            self.tau_model = LassoCV(cv=10, tol=1, random_state=0)
            self.poly = PolynomialFeatures(degree=3, interaction_only=False, include_bias=False)

        elif method == 'nn':
            self.mu0_model = load_model('model_25')
            self.mu1_model = load_model('model_25')
            self.ex_model = load_model('model_ex')
            self.tau_model = load_model('model_25')

        else:
            raise NotImplementedError('Base learner method not specified or typo')

    def fit(self, x, y, w):

        if self.method == 'rf':
            # 1: fit mu_0
            print('Fitting random forest for mu_0')
            self.mu0_model.fit(x[w==0],y[w==0])

            # 2: fit mu_1
            print('Fitting random forest for mu_1')
            self.mu1_model.fit(x[w==1],y[w==1])

            # 3: fit ex
            print('Fitting random forest for e_x')
            self.ex_model.fit(x,w)
            probs = self.ex_model.predict_proba(x)[:,1]
            neg_prob = self.ex_model.predict_proba(x)[:,0]

            # calculate pseudo_outcomes
            mu_w = w * self.mu1_model.predict(x) + (1 - w) * self.mu0_model.predict(x)
            pseudo_outcomes = (w - probs) / (probs * neg_prob) * (y - mu_w) + self.mu1_model.predict(x) - self.mu0_model.predict(x)

            # 4 fit tau
            print('Fitting random forest for tau_x')
            self.tau_model.fit(x,pseudo_outcomes)

        elif self.method == 'lasso':
            x_poly_train = self.poly.fit_transform(x)

            # 1: fit mu_0
            print('Fitting lasso for mu_0')
            self.mu0_model.fit(x_poly_train[w==0],y[w==0])

            # 2: fit mu_1
            print('Fitting lasso for mu_1')
            self.mu1_model.fit(x_poly_train[w==1],y[w==1])

            # 3: fit ex
            print('Fitting lasso for e_x')
            self.ex_model.fit(x_poly_train,w)
            probs = self.ex_model.predict_proba(x_poly_train)[:,1]

            # calculate pseudo_outcomes
            mu_w = w * self.mu1_model.predict(x_poly_train) + (1 - w) * self.mu0_model.predict(x_poly_train)
            pseudo_outcomes = (w - probs) / (probs * (1 - probs)) * (y - mu_w) + self.mu1_model.predict(x_poly_train) - self.mu0_model.predict(x_poly_train)

            # 4 fit tau
            print('Fitting lasso for tau_x')
            self.tau_model.fit(x_poly_train,pseudo_outcomes)

        elif self.method == 'nn':

            # 1: fit mu_0
            print('Training NN for mu_0')
            self.mu0_model.fit(x[w==0],y[w==0],
                               batch_size=100,
                               epochs=100,
                               callbacks=None,
                               verbose=0
                               )

            # 2: fit mu_1
            print('Training NN for mu_1')
            self.mu1_model.fit(x[w==1],y[w==1],
                               batch_size=100,
                               epochs=100,
                               callbacks=None,
                               verbose=0
                               )

            # 3: fit ex
            print('Training NN for e_x')
            self.ex_model.fit(x,w,
                              batch_size=100,
                              epochs=100,
                              callbacks=None,
                              verbose=0
                              )

            probs = np.reshape(keras.activations.sigmoid(self.ex_model.predict(x, verbose=0)),len(x,))

            # calculate pseudo_outcomes
            mu_w = w * self.mu1_model.predict(x, verbose=0) + (1 - w) * self.mu0_model.predict(x, verbose=0)
            pseudo_outcomes = (w - probs) / (probs * (1 - probs)) * (y - mu_w) + self.mu1_model.predict(x, verbose=0) - self.mu0_model.predict(x, verbose=0)

            # 4 fit tau
            print('Training NN for tau_x')
            self.tau_model.fit(x,pseudo_outcomes,
                                   batch_size=100,
                                   epochs=100,
                                   validation_data=None,
                                   callbacks=None,
                                   verbose=0
                                   )

    def predict(self, x):

        if self.method == 'rf':
            predictions = self.tau_model.predict(x)


        elif self.method == 'lasso':
            x_poly_test = self.poly.fit_transform(x)
            predictions = self.tau_model.predict(x_poly_test)

        elif self.method == 'nn':
            predictions = np.reshape(self.tau_model.predict(x, verbose=0),(len(x),))

        else:
            raise NotImplementedError('Base learner method not specified in predict')

        return predictions

In [ ]:
dr_rf = DRLearner('rf')
dr_rf.fit(X_train, Y_train, W_train)
predictions = dr_rf.predict(X_test)
((predictions - tau_test)**2).mean()
# rf: 5.385491721300538 # why different??? ---> because if you take 1 - probs its not exactly the same as taking the [:,0] column!!
# lasso: 6.252082321980517
# nn: 8.35142898943478

# CHECK THIS: CHANGE (1 - PROBS) TO [:,0] TO BE MORE EXACT!!!

# Class RA-Learner

In [10]:
class RALearner():
    def __init__(self, method):
        self.method = method
        if method == 'rf':
            self.mu0_model = RandomForestRegressor(n_estimators=100, max_depth=100, random_state=0)
            self.mu1_model = RandomForestRegressor(n_estimators=100, max_depth=100, random_state=0)
            self.tau_model = RandomForestRegressor(n_estimators=100, max_depth=100, random_state=0)

        elif method == 'lasso':
            self.mu0_model = LassoCV(cv=10, tol=1, random_state=0)
            self.mu1_model = LassoCV(cv=10, tol=1, random_state=0)
            self.tau_model = LassoCV(cv=10, tol=1, random_state=0)
            self.poly = PolynomialFeatures(degree=3, interaction_only=False, include_bias=False)

        elif method == 'nn':
            self.mu0_model = load_model('model_25')
            self.mu1_model = load_model('model_25')
            self.tau_model = load_model('model_25')

        else:
            raise NotImplementedError('Base learner method not specified or typo')

    def fit(self, x, y, w):
        if self.method == 'rf':
            # 1: fit mu_0
            print('Fitting random forest for mu_0')
            self.mu0_model.fit(x[w==0],y[w==0])

            # 2: fit mu_1
            print('Fitting random forest for mu_1')
            self.mu1_model.fit(x[w==1],y[w==1])

            # calculate pseudo_outcomes
            pseudo_outcomes = w*(y - self.mu0_model.predict(x)) + (1 - w)*(self.mu1_model.predict(x) - y)

            # 4 fit tau
            print('Fitting random forest for tau_x')
            self.tau_model.fit(x,pseudo_outcomes)

        elif self.method == 'lasso':
            x_poly_train = self.poly.fit_transform(x)

            # 1: fit mu_0
            print('Fitting lasso for mu_0')
            self.mu0_model.fit(x_poly_train[w==0],y[w==0])

            # 2: fit mu_1
            print('Fitting lasso for mu_1')
            self.mu1_model.fit(x_poly_train[w==1],y[w==1])

            # calculate pseudo_outcomes
            pseudo_outcomes = w*(y - self.mu0_model.predict(x_poly_train)) + (1 - w)*(self.mu1_model.predict(x_poly_train) - y)

            # 4 fit tau
            print('Fitting lasso for tau_x')
            self.tau_model.fit(x_poly_train,pseudo_outcomes)

        elif self.method == 'nn':

            # 1: fit mu_0
            print('Training NN for mu_0')
            self.mu0_model.fit(x[w==0],y[w==0],
                               batch_size=100,
                               epochs=100,
                               callbacks=None,
                               verbose=0
                               )

            # 2: fit mu_1
            print('Training NN for mu_1')
            self.mu1_model.fit(x[w==1],y[w==1],
                               batch_size=100,
                               epochs=100,
                               callbacks=None,
                               verbose=0
                               )

            # calculate pseudo_outcomes
            mu0_predictions = np.reshape(ra_learner_mu0.predict(x, verbose=0),(len(x),))
            mu1_predictions = np.reshape(ra_learner_mu1.predict(x, verbose=0),(len(x),))

            pseudo_outcomes = w*(y - mu0_predictions) + (1 - w)*(mu1_predictions - y)

            # 4 fit tau
            print('Training NN for tau_x')
            self.tau_model.fit(x,pseudo_outcomes,
                                   batch_size=100,
                                   epochs=100,
                                   validation_data=None,
                                   callbacks=None,
                                   verbose=0
                                   )


    def predict(self, x):
        if self.method == 'rf':
            predictions = self.tau_model.predict(x)


        elif self.method == 'lasso':
            x_poly_test = self.poly.fit_transform(x)
            predictions = self.tau_model.predict(x_poly_test)

        elif self.method == 'nn':
            predictions = np.reshape(self.tau_model.predict(x, verbose=0),(len(x),))

        else:
            raise NotImplementedError('Base learner method not specified in predict')

        return predictions

In [14]:
ra_rf = RALearner('nn')
ra_rf.fit(X_train, Y_train, W_train)
predictions = ra_rf.predict(X_test)
((predictions - tau_test)**2).mean()
# rf: 5.355494017645751
# lasso: 8.283890654355236
# nn: 3.3973654461530867

Training NN for mu_0
Training NN for mu_1
Training NN for tau_x


3.3973654461530867

# Class PW-Learner

In [26]:
class PWLearner():
    def __init__(self, method):
        self.method = method
        if method == 'rf':
            self.ex_model = RandomForestClassifier(n_estimators=100, max_depth=100, random_state=0)
            self.tau_model = RandomForestRegressor(n_estimators=100, max_depth=100, random_state=0)

        elif method == 'lasso':
            self.ex_model = LogisticRegressionCV(cv=KFold(10), penalty='l1', solver='saga', tol=1, random_state=0)
            self.tau_model = LassoCV(cv=10, tol=1, random_state=0)
            self.poly = PolynomialFeatures(degree=3, interaction_only=False, include_bias=False)

        elif method == 'nn':
            self.ex_model = load_model('model_ex')
            self.tau_model = load_model('model_25')

        else:
            raise NotImplementedError('Base learner method not specified or typo')


    def fit(self, x, y, w):

        if self.method == 'rf':
            # 3: fit ex
            print('Fitting random forest for e_x')
            self.ex_model.fit(x,w)
            probs = self.ex_model.predict_proba(x)[:,1]
            counter_probs = self.ex_model.predict_proba(x)[:,0]

            # calculate pseudo_outcomes
            pseudo_outcomes = ( w/probs - (1-w)/counter_probs )*y

            # 4 fit tau
            print('Fitting random forest for tau_x')
            self.tau_model.fit(x,pseudo_outcomes)

        elif self.method == 'lasso':
            x_poly_train = self.poly.fit_transform(x)

            # 3: fit ex
            print('Fitting lasso for e_x')
            self.ex_model.fit(x_poly_train,w)

            probs = self.ex_model.predict_proba(x_poly_train)[:,1]
            counter_probs = self.ex_model.predict_proba(x_poly_train)[:,0]

            # calculate pseudo_outcomes
            pseudo_outcomes = ( w/probs - (1-w)/counter_probs )*y

            # 4 fit tau
            print('Fitting lasso for tau_x')
            self.tau_model.fit(x_poly_train,pseudo_outcomes)

        elif self.method == 'nn':

            # 3: fit ex
            print('Training NN for e_x')
            self.ex_model.fit(x,w,
                              batch_size=100,
                              epochs=100,
                              callbacks=None,
                              verbose=0
                              )

            probs = np.reshape(keras.activations.sigmoid(self.ex_model.predict(x, verbose=0)),len(x,))
            counter_probs = 1 - probs

            # calculate pseudo_outcomes
            pseudo_outcomes = ( w/probs - (1-w)/counter_probs )*y

            # 4 fit tau
            print('Training NN for tau_x')
            self.tau_model.fit(x,pseudo_outcomes,
                                   batch_size=100,
                                   epochs=100,
                                   validation_data=None,
                                   callbacks=None,
                                   verbose=0
                                   )

    def predict(self, x):
        if self.method == 'rf':
            predictions = self.tau_model.predict(x)

        elif self.method == 'lasso':
            x_poly_test = self.poly.fit_transform(x)
            predictions = self.tau_model.predict(x_poly_test)

        elif self.method == 'nn':
            predictions = np.reshape(self.tau_model.predict(x, verbose=0),(len(x),))

        else:
            raise NotImplementedError('Base learner method not specified in predict')

        return predictions

In [30]:
pw_rf = PWLearner('nn')
pw_rf.fit(X_train, Y_train, W_train)
predictions = pw_rf.predict(X_test)
((predictions - tau_test)**2).mean()
# rf: 30.529802728890644
# lasso: 16.03059004204301
# nn: 271.8425349295992

Training NN for e_x
Training NN for tau_x


271.8425349295992

# Class U-Learner

In [59]:
class ULearner():
    def __init__(self, method):
        self.method = method
        if method == 'rf':
            self.mux_model = RandomForestRegressor(n_estimators=100, max_depth=100, random_state=0)
            self.ex_model = RandomForestClassifier(n_estimators=100, max_depth=100, random_state=0)
            self.tau_model = RandomForestRegressor(n_estimators=100, max_depth=100, random_state=0)

        elif method == 'lasso':
            self.mux_model = LassoCV(cv=10, tol=1, random_state=0)
            self.ex_model = LogisticRegressionCV(cv=KFold(10), penalty='l1', solver='saga', tol=1, random_state=0)
            self.tau_model = LassoCV(cv=10, tol=1, random_state=0)
            self.poly = PolynomialFeatures(degree=3, interaction_only=False, include_bias=False)

        elif method == 'nn':
            self.mux_model = load_model('model_25')
            self.ex_model = load_model('model_ex')
            self.tau_model = load_model('model_25')

        else:
            raise NotImplementedError('Base learner method not specified or typo')

    def fit(self, x, y, w):

        if self.method == 'rf':
            # 2: fit mu_x
            print('Fitting random forest for mu_x')
            self.mux_model.fit(x,y)

            # 3: fit ex
            print('Fitting random forest for e_x')
            self.ex_model.fit(x,w)
            probs = self.ex_model.predict_proba(x)[:,1]

            # calculate residuals
            residuals = (y - self.mux_model.predict(x))/(w - probs)

            # 4 fit tau
            print('Fitting random forest for tau_x')
            self.tau_model.fit(x,residuals)

        elif self.method == 'lasso':
            x_poly_train = self.poly.fit_transform(x)

            # 2: fit mu_x
            print('Fitting lasso for mu_x')
            self.mux_model.fit(x_poly_train,y)

            # 3: fit ex
            print('Fitting lasso for e_x')
            self.ex_model.fit(x_poly_train,w)
            probs = self.ex_model.predict_proba(x_poly_train)[:,1]

            # calculate pseudo_outcomes
            residuals = (y - self.mux_model.predict(x_poly_train))/(w - probs)

            # 4 fit tau
            print('Fitting lasso for tau_x')
            self.tau_model.fit(x_poly_train,residuals)

        elif self.method == 'nn':

            # 1: fit mu_x
            print('Training NN for mu_x')
            self.mux_model.fit(x,y,
                               batch_size=100,
                               epochs=100,
                               callbacks=None,
                               verbose=0
                               )

            # 3: fit ex
            print('Training NN for e_x')
            self.ex_model.fit(x,w,
                              batch_size=100,
                              epochs=100,
                              callbacks=None,
                              verbose=0
                              )

            probs = np.reshape(keras.activations.sigmoid(self.ex_model.predict(x, verbose=0)),len(x,))

            # calculate pseudo_outcomes
            residuals = (y - self.mux_model.predict(x, verbose=0))/(w - probs)

            # 4 fit tau
            print('Training NN for tau_x')
            self.tau_model.fit(x,residuals,
                                   batch_size=100,
                                   epochs=100,
                                   validation_data=None,
                                   callbacks=None,
                                   verbose=1
                                   )

    def predict(self, x):

        if self.method == 'rf':
            predictions = self.tau_model.predict(x)

        elif self.method == 'lasso':
            x_poly_test = self.poly.fit_transform(x)
            predictions = self.tau_model.predict(x_poly_test)

        elif self.method == 'nn':
            predictions = np.reshape(self.tau_model.predict(x, verbose=0),(len(x),))

        else:
            raise NotImplementedError('Base learner method not specified in predict')

        return predictions

In [61]:
u_rf = ULearner('nn')
u_rf.fit(X_train, Y_train, W_train)
predictions = u_rf.predict(X_test)
((predictions - tau_test)**2).mean()
# rf: 30.921286420155806
# lasso: 7.6762472449663495
# nn: 23.375287707518837 # de isch falsch OBE, da bide class isch korrekt ?????

Training NN for mu_x
Training NN for e_x
Training NN for tau_x
Epoch 1/100
7/7 [==============================] - 0s 2ms/step - loss: 5290.9863 - mean_squared_error: 5290.9863
Epoch 2/100
7/7 [==============================] - 0s 2ms/step - loss: 5267.7207 - mean_squared_error: 5267.7207
Epoch 3/100
7/7 [==============================] - 0s 1ms/step - loss: 5262.9185 - mean_squared_error: 5262.9185
Epoch 4/100
7/7 [==============================] - 0s 2ms/step - loss: 5261.6846 - mean_squared_error: 5261.6846
Epoch 5/100
7/7 [==============================] - 0s 2ms/step - loss: 5260.7236 - mean_squared_error: 5260.7236
Epoch 6/100
7/7 [==============================] - 0s 2ms/step - loss: 5260.0527 - mean_squared_error: 5260.0537
Epoch 7/100
7/7 [==============================] - 0s 2ms/step - loss: 5259.4658 - mean_squared_error: 5259.4658
Epoch 8/100
7/7 [==============================] - 0s 2ms/step - loss: 5258.9658 - mean_squared_error: 5258.9658
Epoch 9/100
7/7 [================

23.375287707518837